In [1]:
# BLOCK 1
# Python Library Imports 
import pandas as pd  # for Dataframes
import numpy as np  # for arrays

import sklearn  # for machine learning
from sklearn.preprocessing import StandardScaler  # for preprocessing 
from sklearn.model_selection import train_test_split  # for spliting the data

import matplotlib.pyplot as plt  # for plotting visuals

import statsmodels.api as sm
import pickle

# Library Setting Changes
%matplotlib inline 
pd.set_option('display.max_columns', None)

In [2]:
# BLOCK 2
#Global Input Variables
csvfile = "unchanged_training_data.csv"  # dataset to be used
df_head_view = 2  # how many rows do you want to view


dataframe_all = pd.read_csv(csvfile, low_memory=False)  # read csvs file # increase memory for large data files
df_shape = dataframe_all.shape

In [11]:
# BLOCK 3
# Data Statistics  
# function that returns: Totals in each category
# return form: [Var Output], [printable Output]
# Params: 
    # dataset= The global variable for the csvfile: dataframe_all
    # column= Feature categorical column of interest, type string 
    # colCompr= The predicted variable in supervised learning, type string
    # categ= Categories in the target feature column, type[StringList]

def statsTotals(dataset, column, colCompr, categ):
    assert(type(dataset) == type(pd.DataFrame())) # dataset = class DataFrame
    assert(type(column) == type(str())) # column = String
    assert(type(colCompr) == type(str())) # colCompr = String
    assert(type(categ) == type(list())) # categ = StringList
    
    # Local VARS #
    num_rows = dataset.shape[0] # get number of rows for data
    num_categs = len(categ) # get how many categories were imputed

    # create a list for each category of the feature column 
    # for each of the categories, how many had "True" predictions vs "False" predictions 
    # var to count for each list
    cntInEachCat = []
    for i in range(num_categs): # create a list for each category in the feature
        cntInEachCat.append(0) # number True in the category 
        cntInEachCat.append(0) # number not True in the category
        cntInEachCat.append(0) # Total Revenue in that category

    # Logic to return Var: cntInEachCat with proper counts of "True/False" predictions
    for row in range(num_rows): # for each data file in file
        for cat in range(num_categs): # For each of the categories the file could be 
            if(dataframe_all.loc[row][column] == categ[cat]): # for each of those categories, if the file is of that category type
                if(dataframe_all.loc[row][colCompr] > 0): # if the comparison row, (e.g. "revenue_30"), is greater than zero
                    cntInEachCat[cat*(num_categs+(3-num_categs))+1] += 1  # add onto the count for that category
                    cntInEachCat[cat*(num_categs+(3-num_categs))+2] += dataframe_all.loc[row][colCompr]  # add in dollars
                else:
                    cntInEachCat[cat*(num_categs+(3-num_categs))] += 1 # add onto the count that there was no revenue
    
    printStatsTotals(num_categs, cntInEachCat, categ, column)
    return cntInEachCat  # if I want the numbers for anything


In [21]:
# BLOCK 4
# Print Stats Totals
def printStatsTotals(num_categs, cntInEachCat, categ, column):
    print(column, categ)
    printlist = []
    accountedfor = 0
    for cat in range(num_categs):
        total = cntInEachCat[cat*(num_categs+(3-num_categs))] + cntInEachCat[cat*(num_categs+(3-num_categs))+1]        
        trueFalse = (cntInEachCat[cat*(num_categs+(3-num_categs))+1], "to", cntInEachCat[cat*(num_categs+(3-num_categs))])
        try:
            pcntTrueVsTotal = cntInEachCat[cat*(num_categs+(3-num_categs))+1] / total
        except ZeroDivisionError:
             pcntTrueVsTotal = "division by zero error"
        howMuchTotal = cntInEachCat[cat*(num_categs+(3-num_categs))+2]
        try:
            howMuchPerCatAvg = cntInEachCat[cat*(num_categs+(3-num_categs))+2] / cntInEachCat[cat*(num_categs+(3-num_categs))+1]
        except ZeroDivisionError:
             howMuchPerCatAvg = "division by zero error"

        print(categ[cat], "Total:", total)
        print(categ[cat], "True/False:", trueFalse)
        print(categ[cat], "% True of Total:", pcntTrueVsTotal)
        print(categ[cat], "HowMuch Total:", howMuchTotal)
        print(categ[cat], "HowMuch per True:", howMuchPerCatAvg)
        
        addon = categ[cat] + " (" +"F="+ str(trueFalse[2]) + ", T=" + str(trueFalse[0]) +", Rev="+ str(howMuchTotal) +")"
        accountedfor += total
        printlist.append(addon)
    printlist.append(("Total: "+ str(accountedfor)))
    print(printlist)
    print()

In [5]:
# BLOCK 5
# Data Visualization of True data sets
def dataCollection(dataset, column, colCompr, categ, override):
    assert(type(dataset) == type(pd.DataFrame())) # dataset = class DataFrame
    assert(type(column) == type(str())) # column = String
    assert(type(colCompr) == type(str())) # colCompr = String
    assert(type(categ) == type(list())) # categ = (String) List
    
    # Local VARS #
    num_rows = dataset.shape[0] # get number of rows for data
    num_categs = len(categ) # get how many categories were imputed

    # create a list to hold the counts
    # for each of the categories, how many are in that category? How many were "True" in that category compared to "False"?
    revLists = []
    labels = ""
    for i in range(num_categs):
        revLists.append([]) # number True in the category 
        labels = labels +" " + categ[i]
    
    if(override):
#         # Logic to return Var: cntInEachCat = [False, True, How much] * number of categories    
        for row in range(num_rows): # for each row in file 
            for cat in range(num_categs): # For each of the categories the file could be 
                if(dataframe_all.loc[row][column] == categ[cat]): # for each of those categories, if the file is of that category type
                    if(dataframe_all.loc[row][colCompr] > 0): # if the comparison row, (e.g. "revenue_30"), is greater than zero
                        revLists[cat].append(dataframe_all.loc[row][colCompr])

        # Save query data to file                
        filename = column + "-" + colCompr + "-" + "IndividualRevenues.sav"
#         print(revLists)
        pickle.dump(revLists, open(filename, 'wb'))
        print(filename)
    else:
        # Get queryed data from file
        filename = column + "-" + colCompr + "-" + "IndividualRevenues.sav"
        revLists = pickle.load(open(filename, 'rb'))
        print("FILENAME:", filename)
        graphDataCollection(column, categ, revLists)
        return filename 
    

In [6]:
# BLOCK 6

def graphDataCollection(column, categ, revLists):      
    # Graphing #    
    # 5/1 if category doesnt have more than ~linearError
    f1, ax1 = plt.subplots()
    ax1.set_title(column + ": " + 'Boxplot Outlier Detector Visual')
    ax1.boxplot(revLists, labels=categ, notch=True, vert = False, showfliers = True)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    sm.graphics.beanplot(revLists, ax=ax, labels=categ)
    ax.set_xlabel(column.upper())
    ax.set_ylabel("Revenue $")

    plt.show()

In [7]:
# BLOCK 7
# Data Cleaning


In [9]:
# BLOCK 8
# Data Splitting







# Load the Diabetes dataset
# columns = “age sex bmi map tc ldl hdl tch ltg glu”.split() # Declare the columns names
# diabetes = datasets.load_diabetes() # Call the diabetes dataset from sklearn
# df = pd.DataFrame(diabetes.data, columns=columns) # load the dataset as a pandas data frame
# y = diabetes.target # define the target variable (dependent variable) as y

# # create training and testing vars
# X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
# print X_train.shape, y_train.shape
# print X_test.shape, y_test.shape

In [24]:
# MAIN

# print("df_shape:", df_shape)
# print(dataframe_all.head(df_head_view))


# roll_up = statsTotals(dataframe_all, "roll_up", "revenue_30", ["Onboarding", "Unmanaged", "Retention"])
# currentstatus = statsTotals(dataframe_all, "currentstatus", "revenue_30", ["Active", "Enrolled"])
# companytypegroup = statsTotals(dataframe_all, "companytypegroup", "revenue_30", ["Business", "Trade"])
# team = statsTotals(dataframe_all, "team", "revenue_30", ["US", "CA"])
# customersource = statsTotals(dataframe_all, "customersource", "revenue_30", 
#                              ["Internal Application", "External Application", "Search - Paid", "Internal Customer Scrape", 
#                              "Affiliates", "Email", "Other", "Gateway", "Social - Paid", "Quotes"])
# accrole = statsTotals(dataframe_all, "accrole", "revenue_30", ["None", "Primary", "Purchaser"])
# num_employees = statsTotals(dataframe_all, "num_employees", "revenue_30", ["None", "2to5", "6to10", "11to50", "50plus"])
# num_purchases_year = statsTotals(dataframe_all, "num_purchases_year", "revenue_30", ["None", "1to2", "3to5", "6to10", "11to25", "25plus"])
cost_purchases_year = statsTotals(dataframe_all, "cost_purchases_year", "revenue_30", ["None", "lessthan1", "1to5", "5to25","25to100", "100plus"])
# enrollmentmethod = statsTotals(dataframe_all, "enrollmentmethod", "revenue_30", ["directEIN", "directOther", "other", "email", "phone", "liveTransfer"])


# var = dataCollection(dataframe_all, "roll_up", "revenue_30", ["Onboarding", "Unmanaged", "Retention"], True)
# var1 = dataCollection(dataframe_all, "roll_up", "revenue_30", ["Onboarding", "Unmanaged", "Retention"], False)
# var = statsTotals(dataframe_all, "currentstatus", "revenue_30", ["Active", "Enrolled"])
# print(var)

cost_purchases_year ['None', 'lessthan1', '1to5', '5to25', '25to100', '100plus']
None Total: 21991
None True/False: (2083, 'to', 19908)
None % True of Total: 0.09472056750488836
None HowMuch Total: 1486059.82299999
None HowMuch per True: 713.422862698027
lessthan1 Total: 893
lessthan1 True/False: (93, 'to', 800)
lessthan1 % True of Total: 0.10414333706606943
lessthan1 HowMuch Total: 33172.02230000002
lessthan1 HowMuch per True: 356.68841182795717
1to5 Total: 2360
1to5 True/False: (336, 'to', 2024)
1to5 % True of Total: 0.1423728813559322
1to5 HowMuch Total: 191465.86450000014
1to5 HowMuch per True: 569.8388824404766
5to25 Total: 1865
5to25 True/False: (256, 'to', 1609)
5to25 % True of Total: 0.13726541554959787
5to25 HowMuch Total: 238450.2355
5to25 HowMuch per True: 931.446232421875
25to100 Total: 682
25to100 True/False: (101, 'to', 581)
25to100 % True of Total: 0.14809384164222875
25to100 HowMuch Total: 121622.23640000001
25to100 HowMuch per True: 1204.1805584158417
100plus Total: 33

In [13]:
# Optimizations: 
#     remove outliers
print(9918+18199)
print(9986+14748+3392)
print(22748+5378)
print(10888+3032+2113+5849+1523+105+904+19+3203+20)

28117
28126
28126
27656
